In [18]:
import os
import numpy as np
import random
random.seed = 42
from path import Path
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier  
import ot

import sys
sys.path.append("../src")
from utility import HCP


In [19]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces


def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
        layout=go.Layout(
            updatemenus=[dict(type='buttons',
                showactive=False,
                y=1,
                x=0.8,
                xanchor='left',
                yanchor='bottom',
                pad=dict(t=45, r=10),
                buttons=[dict(label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=50, redraw=True),
                        transition=dict(duration=0),
                        fromcurrent=True,
                        mode='immediate'
                        )]
                    )
                ])]
        ),
        frames=frames
    )

    return fig


def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

In [20]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):

        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points


class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [21]:
path = Path("./ModelNet10")

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
 
print(classes)

data = pd.read_csv('metadata_modelnet10.csv')
data.columns = ['object_id', 'type', 'split', 'object_path']
data.head(5)

{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


,object_id,type,split,object_path
0,bathtub_0107,bathtub,test,bathtub/test/bathtub_0107.off
1,bathtub_0108,bathtub,test,bathtub/test/bathtub_0108.off
2,bathtub_0109,bathtub,test,bathtub/test/bathtub_0109.off
3,bathtub_0110,bathtub,test,bathtub/test/bathtub_0110.off
4,bathtub_0111,bathtub,test,bathtub/test/bathtub_0111.off


In [22]:
print(np.sum(data.split=='train'))
train_data = data[data['split']=='train'].reset_index(drop=True)
print(np.sum(data.split=='test'))
test_data = data[data['split']=='test'].reset_index(drop=True)

3991
908


In [23]:
seed = 0
np.random.seed(seed)
subclass = classes.keys()

Ktrain = 50*10
Ktest = 30*10

train_3D = [None]*Ktrain
train_label = np.zeros(Ktrain)
i = 0
j = 0

KK = 500

for category in subclass:

    temple_data = train_data[train_data['type']==category].sample(n=int(Ktrain/10),random_state=seed).reset_index(drop=True)

    for k in range(temple_data.shape[0]):
        temple_path = temple_data.object_path[k]
        with open(path/temple_path, 'r') as f:
            verts, faces = read_off(f)
            pointcloud = PointSampler(KK)((verts, faces))
            train_3D[i] = Normalize()(pointcloud)
            train_label[i] = j
            i += 1
    
    print(k+1)
    j += 1
    print(i)


test_3D = [None]*Ktest
test_label = np.zeros(Ktest)
i = 0
j = 0

for category in subclass:

    temple_data = test_data[test_data['type']==category].sample(n=int(Ktest/10),random_state=seed).reset_index(drop=True)

    for k in range(temple_data.shape[0]):
        temple_path = temple_data.object_path[k]
        with open(path/temple_path, 'r') as f:
            verts, faces = read_off(f)
            pointcloud = PointSampler(KK)((verts, faces))
            test_3D[i] = Normalize()(pointcloud)
            test_label[i] = j
        i = i+1
        
    print(k+1)
    j += 1
    print(i)

50
50
50
100
50
150
50
200
50
250
50
300
50
350
50
400
50
450
50
500
30
30
30
60
30
90
30
120
30
150
30
180
30
210
30
240
30
270
30
300


In [24]:
import time
t = time.time()

np.random.seed(seed)
hcp_dist_matrix = np.zeros((Ktrain,Ktrain))
for i in range(Ktrain):
    for j in range(i+1,Ktrain):
        hcp_dist_matrix[i,j] = HCP(train_3D[i],train_3D[j])
hcp_dist_matrix = hcp_dist_matrix+hcp_dist_matrix.T

hcp_dist_matrix2 = np.zeros((Ktest,Ktrain))
for i in range(Ktest):
    for j in range(Ktrain):
        hcp_dist_matrix2[i,j] = HCP(test_3D[i],train_3D[j])

estimator = KNeighborsClassifier(metric='precomputed',n_neighbors=5)

estimator.fit(hcp_dist_matrix,train_label)
print('Accuracy is ', np.sum(estimator.predict(hcp_dist_matrix2)==test_label)/Ktest, '%')

print('Time is ', time.time()-t, 'seconds')

Accuracy is  0.8433333333333334 %
Time is  95.83503890037537 seconds
